In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('C:/Users/lenovo/Desktop/资料/暑期资料/场外衍生品/对冲PnL.xlsx')

In [3]:
contracts = data[['position_id','market_value','delta_cash','valuation_date']].copy()
contracts['valuation_date'] = contracts['valuation_date'].apply(lambda x:str(x)[:10])   #日期时间戳格式全部改为字符串格式，方便后面操作
contracts

,position_id,market_value,delta_cash,valuation_date
0,XNZQ-CW-0002_0_0,461643.0935,-3.038818e+07,2021-11-15
1,XNZQ-CW-0001_0,366510.3351,-1.482559e+07,2021-11-15
2,test19-005_1,14957.8280,-1.707275e+05,2021-11-24
3,GLSC-SWSC-0004_0,129908.2398,4.606313e+06,2021-11-24
4,testpos1_0,0.0000,NaN,2021-11-24
...,...,...,...,...
17101,SUX705_FTP_0,564480.3900,-5.328242e+06,2022-07-26
17102,SWHY-1010047-22004_0,0.0000,NaN,2022-07-26
17103,SUD064_0,0.0000,NaN,2022-07-26
17104,SWC153_0,0.0000,NaN,2022-07-26


In [4]:
id_l = list(contracts['position_id'].unique())  #合约id去重,得到所有id列表

#单个id时间序列
id_contracts = contracts[contracts['position_id'] == id_l[0]]

#产品端PnL
product_pnl = (id_contracts['market_value'].shift(-1) - id_contracts['market_value'])[:-1].to_frame()  
product_pnl.rename(columns={'market_value':'product_pnl'},inplace=True)
date_col = id_contracts['valuation_date'].shift(-1)[:-1]   #时间列
date_col.name = 'date'
product_pnl.index=date_col  #索引改为日期
product_pnl

,product_pnl
date,
2021-11-24,-7.699682e+05
2021-11-23,-1.882517e+04
2021-11-25,-1.821671e+06
2021-11-26,1.936374e+06
2021-11-29,-4.951861e+04
...,...
2022-05-26,-5.985393e+05
2022-05-27,9.053518e+04
2022-05-30,-3.945175e+05


In [6]:
#ic合约数据
ic_futures = pd.read_excel('C:/Users/lenovo/Desktop/资料/暑期资料/量化/历史数据/IC期货.xlsx')
ic_futures['日期'] = ic_futures['日期'].apply(lambda x: str(x)[:10])   #日期时间戳格式全部改为字符串格式，方便后面操作
ic_futures.rename(columns={'日期':'valuation_date'},inplace=True)
ic_futures = ic_futures.set_index('valuation_date')
ic_futures

,收盘价（主力合约）
valuation_date,
2022-07-27,6298.0
2022-07-26,6273.4
2022-07-25,6173.6
2022-07-22,6244.8
2022-07-21,6310.2
...,...
2015-04-22,8189.8
2015-04-21,8140.4
2015-04-20,7622.2


In [7]:
#对冲端delta cash
delta_df = id_contracts[['delta_cash','valuation_date']]
delta_df = delta_df.set_index('valuation_date')
delta_df = delta_df.join(ic_futures)   #连接上对应日期的IC期货主力合约收盘价
delta_df

,delta_cash,收盘价（主力合约）
valuation_date,,
2021-11-15,-3.038818e+07,7104.8
2021-11-24,-2.515821e+07,7221.4
2021-11-23,-2.555668e+07,7225.0
2021-11-25,-9.196564e+04,7218.0
2021-11-26,-2.541881e+07,7196.4
...,...,...
2022-05-26,-5.156537e+07,5852.8
2022-05-27,-5.128499e+07,5845.0
2022-05-30,-5.238547e+07,5872.4


In [39]:
#对冲端盈亏
hedging_df = delta_df.copy()
hedging_df['单日对冲手数'] = -round((hedging_df['delta_cash']/(hedging_df['收盘价（主力合约）']*200)))
hedging_df['ic期货仓位'] = hedging_df['单日对冲手数'].cumsum()
hedging_df['单日对冲成本'] = hedging_df['单日对冲手数']*hedging_df['收盘价（主力合约）']*200   #对冲成本：正，表示买入；负，表示卖出
hedging_df['累计成本']=hedging_df['单日对冲成本'].cumsum()
hedging_df['已实现盈亏'] = -hedging_df['单日对冲成本']
hedging_df['一手成本'] = hedging_df['累计成本']/hedging_df['ic期货仓位']
# hedging_df['浮动盈亏'] = hedging_df['ic期货仓位']*(200*hedging_df['收盘价（主力合约）'] - hedging_df['一手成本'])
hedging_df['浮动盈亏'] = 200*hedging_df['收盘价（主力合约）']*hedging_df['ic期货仓位'] - hedging_df['累计成本']
hedging_df.index.name = 'date'
hedging_df

,delta_cash,收盘价（主力合约）,单日对冲手数,ic期货仓位,单日对冲成本,累计成本,已实现盈亏,一手成本,浮动盈亏
date,,,,,,,,,
2021-11-15,-3.038818e+07,7104.8,21.0,21.0,29840160.0,2.984016e+07,-29840160.0,1.420960e+06,-3.725290e-09
2021-11-24,-2.515821e+07,7221.4,17.0,38.0,24552760.0,5.439292e+07,-24552760.0,1.431393e+06,4.897200e+05
2021-11-23,-2.555668e+07,7225.0,18.0,56.0,26010000.0,8.040292e+07,-26010000.0,1.435766e+06,5.170800e+05
2021-11-25,-9.196564e+04,7218.0,0.0,56.0,0.0,8.040292e+07,-0.0,1.435766e+06,4.386800e+05
2021-11-26,-2.541881e+07,7196.4,18.0,74.0,25907040.0,1.063100e+08,-25907040.0,1.436621e+06,1.967600e+05
...,...,...,...,...,...,...,...,...,...
2022-05-26,-5.156537e+07,5852.8,44.0,3415.0,51504640.0,4.301884e+09,-51504640.0,1.259703e+06,-3.044216e+08
2022-05-27,-5.128499e+07,5845.0,44.0,3459.0,51436000.0,4.353320e+09,-51436000.0,1.258549e+06,-3.097490e+08
2022-05-30,-5.238547e+07,5872.4,45.0,3504.0,52851600.0,4.406172e+09,-52851600.0,1.257469e+06,-2.907937e+08


In [42]:
#产品端盈亏+对冲端盈亏 = 总盈亏
pnl_df = hedging_df[['已实现盈亏','浮动盈亏']].join(product_pnl)
pnl_df.rename(columns={'已实现盈亏': '已实现盈亏'},inplace=True)
pnl_df.fillna(0,inplace=True)
pnl_df['总pnl'] = pnl_df['已实现盈亏']+pnl_df['浮动盈亏']+pnl_df['product_pnl']
pnl_df

,已实现盈亏,浮动盈亏,product_pnl,总pnl
date,,,,
2021-11-15,-29840160.0,-3.725290e-09,0.000000e+00,-2.984016e+07
2021-11-24,-24552760.0,4.897200e+05,-7.699682e+05,-2.483301e+07
2021-11-23,-26010000.0,5.170800e+05,-1.882517e+04,-2.551175e+07
2021-11-25,-0.0,4.386800e+05,-1.821671e+06,-1.382991e+06
2021-11-26,-25907040.0,1.967600e+05,1.936374e+06,-2.377391e+07
...,...,...,...,...
2022-05-26,-51504640.0,-3.044216e+08,-5.985393e+05,-3.565248e+08
2022-05-27,-51436000.0,-3.097490e+08,9.053518e+04,-3.610945e+08
2022-05-30,-52851600.0,-2.907937e+08,-3.945175e+05,-3.440398e+08
